In [1]:
using ModelingToolkit, JuMP, EOptInterface
using ModelingToolkit: t_nounits as t, D_nounits as D

@mtkmodel KineticParameterEstimation begin
    @parameters begin
        T = 273
        K_2 = 46*exp(6500/T-18)
        K_3 = 2*K_2
        k_1 = 53
        k_1s = k_1*1e-6
        k_5 = 1.2e-3
        c_O2 = 2e-3

        k_2f
        k_3f
        k_4
    end
    @variables begin
        x_A(t) = 0.0
        x_B(t) = 0.0
        x_D(t) = 0.0
        x_Y(t) = 0.4
        x_Z(t) = 140.0
        I(t)
    end
    @equations begin
        D(x_A) ~ k_1*x_Z*x_Y - c_O2*(k_2f + k_3f)*x_A + k_2f/K_2*x_D + k_3f/K_3*x_B - k_5*x_A^2
        D(x_B) ~ c_O2*k_3f*x_A - (k_3f/K_3 + k_4)*x_B
        D(x_D) ~ c_O2*k_2f*x_A - k_2f/K_2*x_D
        D(x_Y) ~ -k_1s*x_Z*x_Y
        D(x_Z) ~ -k_1*x_Z*x_Y
        I ~ x_A + 2/21*x_B + 2/21*x_D
    end
end

@mtkcompile o = KineticParameterEstimation()

tspan = (0.0,2.0)
tstep = 0.01
include("kinetic_intensity_data.jl")
intensity(x_A,x_B,x_D) = x_A + 2/21*x_B + 2/21*x_D

intensity (generic function with 1 method)

In [2]:
using EAGO, Gurobi
@mtkcompile o = KineticParameterEstimation()
factory = () -> EAGO.Optimizer(SubSolvers(; r = Gurobi.Optimizer()))
model = Model(factory)
# trying a bunch of different settings, this at least allows it to run for a bit
set_attribute(model, "time_limit", 3600*24.0)
set_attribute(model, "relative_tolerance", 1e-4)
set_attribute(model, "output_iterations", 1)
set_attribute(model, "obbt_repetitions", 5)
set_attribute(model, "fbbt_lp_repetitions", 5)
set_attribute(model, "branch_cvx_factor", 0.5)
set_attribute(model, "absolute_constraint_feas_tolerance", 1e-5)
set_attribute(model, "branch_pseudocost_on", true)
set_attribute(model, "reverse_subgrad_tighten", true)
set_attribute(model, "cut_max_iterations", 12)
set_attribute(model, "verbosity", 2)
decision_vars(o) # Displays: x_Z(t), x_Y(t), x_D(t), x_B(t), x_A(t), k_2f, k_3f, k_4
# FIRST, create discretized state decision variables
N = Int(floor((tspan[2] - tspan[1])/tstep))+1
V = length(unknowns(o))
@variable(model, -70 <= z[1:V,1:N] <= 150.0 ) # ̇z = (x_Z(t), x_Y(t), x_D(t), x_B(t), x_A(t))
# SECOND, create free design decision variables
pL = [10, 10, 0.001]
pU = [1200, 1200, 40]
@variable(model, pL[i] <= p[i=1:3] <= pU[i]) # p = (k_2f, k_3f, k_4)
register_odesystem(model, o, tspan, tstep, "IE")
@objective(model, Min, sum((intensity(z[5,i],z[4,i],z[3,i]) - data[i-1])^2 for i in 2:N))

Set parameter Username
Set parameter LicenseID to value 2617239
Academic license - for non-commercial use only - expires 2026-02-02


z[5,2]² + 0.19047619047619047 z[4,2]*z[5,2] + 0.19047619047619047 z[3,2]*z[5,2] + 0.009070294784580497 z[4,2]² + 0.018140589569160995 z[3,2]*z[4,2] + 0.009070294784580497 z[3,2]² + z[5,3]² + 0.19047619047619047 z[4,3]*z[5,3] + 0.19047619047619047 z[3,3]*z[5,3] + 0.009070294784580497 z[4,3]² + 0.018140589569160995 z[3,3]*z[4,3] + 0.009070294784580497 z[3,3]² + z[5,4]² + 0.19047619047619047 z[4,4]*z[5,4] + 0.19047619047619047 z[3,4]*z[5,4] + 0.009070294784580497 z[4,4]² + 0.018140589569160995 z[3,4]*z[4,4] + 0.009070294784580497 z[3,4]² + z[5,5]² + 0.19047619047619047 z[4,5]*z[5,5] + 0.19047619047619047 z[3,5]*z[5,5] + 0.009070294784580497 z[4,5]² + 0.018140589569160995 z[3,5]*z[4,5] + 0.009070294784580497 z[3,5]² + z[5,6]² + 0.19047619047619047 z[4,6]*z[5,6] + 0.19047619047619047 z[3,6]*z[5,6] + 0.009070294784580497 z[4,6]² + 0.018140589569160995 z[3,6]*z[4,6] + 0.009070294784580497 z[3,6]² + [[...1140 terms omitted...]] + z[5,197]² + 0.19047619047619047 z[4,197]*z[5,197] + 0.1904761904

In [ ]:
JuMP.optimize!(model)

 
Lower Bound (First Iteration): -1.2116986805201499e6
Solution: [139.0, 0.4, 0.0, 0.0, 0.0, 115.5173907466067, 0.3999856077944809, 0.37330542263419086, 0.23533029195395416, 24.126773688127905, 95.33917006282739, 0.3999754395408871, 1.0528463804903287, 0.4722096659708408, 43.66994351620936, 78.58424387438049, 0.39996877409547577, 1.984712309075538, 1.1679671344410423, 59.41403767048911, 64.88043132872137, 0.39996502823862884, 3.1296206517008476, 1.7765989803778082, 72.04637461965001, 53.564184768808964, 0.39996369761748446, 4.452355639259452, 3.0824222744931884, 82.12532866797265, 44.21272902428351, 0.399964369488797, 5.935619190378525, 6.282016562031906, 90.12712196306288, 36.50988156171027, 0.3999667020390109, 7.542953802877947, 11.392097016652755, 96.40922914186241, 30.183201143329846, 0.3999502984563146, 9.2641291120704, 16.503087883276947, 101.30013834657115, 24.998027664029525, 0.39993494922202455, 11.015335282633473, 21.614770595087634, 105.09825933517935, 20.703043457328732, 0.

┌ Warning: EAGO lacks a specialized configuration routine for the subsolver (Gurobi)
│ you selected. As a result, EAGO cannot set the subsolver tolerances based on the
│ absolute_tolerance, relative_tolerance, and absolute_constraint_feas_tolerance
│ parameters passed to the EAGO optimizer. Consequently, you need to ensure that the tolerances
│ set in the provided subsolver are appropriate (for instance if the absolute_tolerance = 1E-3
│ then the absolute tolerance for a subsolver should be < 1E-4 and any feasibility tolerances
│ should be as conservative as the absolute_constraint_feas_tolerance). If you see this message
│ please submit an issue at https://github.com/PSORLab/EAGO.jl/issues/new/choose requesting
│ that a configuration routine be added for this subsolver.
└ @ EAGO C:\Users\josep\.julia\packages\EAGO\WUXx0\src\eago_optimizer\optimize\nonconvex\configure_subsolver.jl:17



Feasibility: true
Termination Status Code: OPTIMAL
Result Code: FEASIBLE_POINT
 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

 
Upper Bound: 16796.04904093112
Solution: [140.0, 0.4, -8.366287168389926e-35, 1.5747318042033624e-29, -9.07339131303667e-30, 115.51278807808937, 0.39997551278807814, 0.38127845528011883, 0.17932157330814633, 23.89648870148108, 95.30944354544975, 0.39995530944354546, 1.0675868264194928, 0.48157991232300473, 43.02655614808785, 78.6402648339408, 0.39993864026483394, 1.9961607758734463, 0.863196126348354, 58.23283437610648, 64.88683826517875, 0.3999248868382652, 3.115395808141557, 1.290835021711255, 70.21271654840153, 53.539019

Excessive output truncated after 544547 bytes.

In [ ]:
println("STATUS: $(JuMP.termination_status(model)), RESULT CODE: $(JuMP.primal_status(model))")
println("TIME: $(round.(JuMP.solve_time(model),digits=6))")
println("f^* = $(round(JuMP.objective_value(model),digits=6))")
println("p* = $(round.(JuMP.value.(p),digits=5)).")